In [1]:
import os

In [2]:
pwd

'c:\\Users\\USER\\Downloads\\khushboo\\Udemy\\Projects\\Chicken-Disease-Classification-DL\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\USER\\Downloads\\khushboo\\Udemy\\Projects\\Chicken-Disease-Classification-DL'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
import cnnClassifier.constants.__init__ as constant 
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = constant.CONFIG_FILE_PATH,
        params_filepath = constant.PARAMS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [7]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-22 15:05:03,114: INFO: common: yaml file: config\config.yaml loaded successfuly]
[2024-04-22 15:05:03,118: INFO: common: yaml file: params.yml loaded successfuly]
[2024-04-22 15:05:03,119: INFO: common: Created directory at: artifacts]
[2024-04-22 15:05:03,120: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-04-22 15:05:15,521: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FDA0:33ED63:1C2CFE:263741:66262F49
Accept-Ranges: bytes
Date: Mon, 22 Apr 2024 09:35:06 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10223-MAA
X-Cache: MIS